# Props Data Organizer

## Definitions of Formulas Used in This Notebook

### Estimated Value (EV)
EV is the average amount you can expect to win or lose per bet if you placed the same bet many times. It helps identify profitable betting opportunities by comparing the expected return to the risk involved.

**Formula:**
$$
\text{EV} = (\text{Probability of Winning} \times \text{Profit if Win}) - (\text{Probability of Losing} \times \text{Loss if Lose})
$$

### Kelly Criterion
The Kelly Criterion is a formula used to determine the optimal size of a series of bets. It aims to maximize the logarithm of wealth, balancing the trade-off between risk and reward. The formula considers both the probability of winning and the odds offered, guiding you on how much of your bankroll to wager on each bet.

**Formula:**
$$
\text{Kelly Fraction} = \frac{(\text{Probability of Winning} \times (\text{Odds} + 1)) - 1}{\text{Odds}}
$$

In [2]:
import pandas as pd 
import numpy as np
import time
import requests
from NBAData.gambling import *
from datetime import datetime

today = datetime.now()
formatted_date = today.strftime("%m_%d_%y")

### Historical data from the odds api, but i have to pay. Worst case ill pay, if im not able to scrap from betting pros

In [2]:
base_url = "https://api.the-odds-api.com/v4/sports/basketball_nba/events/"
API_KEY = '8aa8bb1fc5d56c98b45d6f3f58beab70'
Sport = 'basketball_nba'
Regions = 'us'
Market = 'h2h,odds'
Odds_format = 'american'
Date_format = 'iso' 
date = '2021-10-22T22:45:00Z'
id = '34e829d65b6a0cbb49573338a86772ec'

url = (
    f"https://api.the-odds-api.com/v4/historical/sports/{Sport}/events"
    f"?apiKey={API_KEY}"
    f"&date={date}"
    f"&regions={Regions}"
)

response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to get events: {response.status_code}, {response.text}")
else:
    events = response.json()
    for event in events:
        print(event['id'], event['commence_time'], event['home_team'], event['away_team'])

Failed to get events: 401, {"message":"Historical odds are only available on paid usage plans. See usage plans at https://the-odds-api.com","error_code":"HISTORICAL_UNAVAILABLE_ON_FREE_USAGE_PLAN","details_url":"https://the-odds-api.com/liveapi/guides/v4/api-error-codes.html#historical-unavailable-on-free-usage-plan"}



### Grabs players odds for the day (US all boookmakers, DFS is prizepicks and underdogs)

In [3]:
from NBAPropFinder.NBAPropFinder import NBAPropFinder

nba_props = NBAPropFinder()
prizePicks = nba_props.dataframe
prizePicks.to_csv(f'PROPS_DATA/Playoffs_US({formatted_date}).csv')
prizePicks.head(10)

Scraping Odds API...
Organizing Data...


,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,FanDuel,player_points,Pascal Siakam,Over,20.5,-102
1,FanDuel,player_points,Pascal Siakam,Under,20.5,-125
2,FanDuel,player_points,Tyrese Haliburton,Over,15.5,-115
3,FanDuel,player_points,Tyrese Haliburton,Under,15.5,-111
4,FanDuel,player_points,Luguentz Dort,Over,7.5,-130
5,FanDuel,player_points,Luguentz Dort,Under,7.5,102
6,FanDuel,player_points,T.J. McConnell,Over,9.5,-120
7,FanDuel,player_points,T.J. McConnell,Under,9.5,-106
8,FanDuel,player_points,Jalen Williams,Over,21.5,-125
9,FanDuel,player_points,Jalen Williams,Under,21.5,-102


### Single Bets from bookmakers that dont include prizePicks or UnderDogs

In [4]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
all_bookmakers = pd.read_csv(f'./PROPS_DATA/Playoffs_US({formatted_date}).csv')
prop_type = all_bookmakers['CATEGORY'].unique()
print(prop_type)
all_bookmakers

['player_points' 'player_rebounds' 'player_assists' 'player_threes'
 'player_blocks' 'player_steals' 'player_points_rebounds_assists'
 'player_points_rebounds' 'player_points_assists'
 'player_rebounds_assists' 'player_turnovers' 'player_blocks_steals']


,Unnamed: 0,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,0,FanDuel,player_points,Pascal Siakam,Over,20.5,-102
1,1,FanDuel,player_points,Pascal Siakam,Under,20.5,-125
2,2,FanDuel,player_points,Tyrese Haliburton,Over,15.5,-115
3,3,FanDuel,player_points,Tyrese Haliburton,Under,15.5,-111
4,4,FanDuel,player_points,Luguentz Dort,Over,7.5,-130
...,...,...,...,...,...,...,...
1888,1888,DraftKings,player_blocks_steals,Isaiah Hartenstein,Under,1.5,-200
1889,1889,DraftKings,player_blocks_steals,Obi Toppin,Over,1.5,165
1890,1890,DraftKings,player_blocks_steals,Obi Toppin,Under,1.5,-215
1891,1891,DraftKings,player_blocks_steals,T.J. McConnell,Over,1.5,165


In [5]:
import scipy.stats as stats


def single_bet(data, bookmakers, category='player_points', stat_line='PTS'):  
    Props = bookmakers[['NAME', 'BOOKMAKER', 'CATEGORY', 'LINE', 'OVER/UNDER', 'ODDS']].loc[bookmakers['CATEGORY'] == category]
    results = []

    for idx, row in Props.iterrows():
        name = row['NAME']
        bookmaker = row['BOOKMAKER']
        line = row['LINE']
        over_under = row['OVER/UNDER']
        odds = row['ODDS']

        # Get combined or single stat values
        stat_values = get_combined_stat_values(data, name, stat_line)
        if stat_values.empty:
            continue

        mean = round(stat_values.mean(), 2)
        std = round(stat_values.std(), 2)
        if std == 0:
            continue

        # Z-score-based probability
        z_score = (line - mean) / std
        norm_prob = 1 - stats.norm.cdf(z_score)
        if over_under == "Under":
            norm_prob = 1 - norm_prob

        # Monte Carlo simulation
        sim_points = np.random.normal(mean, std, 10000)
        sim_prob = np.mean(sim_points > line)
        if over_under == "Under":
            sim_prob = 1 - sim_prob

        # Weighted average of sim and norm prob
        final_prob = 0.8 * sim_prob + 0.2 * norm_prob
        
        # EV calculation
        stake = 100
        profit = (odds / 100) * stake if odds > 0 else (100 / abs(odds)) * stake
        payout = stake + profit
        ev = (final_prob * profit) - ((1 - final_prob) * stake)

        kelly = kelly_criterion(final_prob, payout, stake)

        # Calculate fair odds
        try:
            fair_odds = fairProb(bookmakers, name, line, category, over_under)
        except ValueError as e:
            fair_odds = None

        results.append({
            'NAME': name,
            'BOOKMAKER': bookmaker,
            'CATEGORY': category,
            'LINE': line,
            'OVER/UNDER': over_under,
            'ODDS': odds,
            'FAIR ODDS': fair_odds,
            'SIM PROB': round(final_prob, 3),
            'EV': round(ev, 2),
            'KELLY CRITERION': kelly,
        })

    return pd.DataFrame(results)

In [6]:
# Dictionary mapping prop categories to their stat columns
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_threes': 'FG3M',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_field_goals': 'FGM',
    'player_threes': 'FG3M',
    'player_frees_made': 'FTM',
    'player_frees_attempts': 'FTA',
    'player_turnovers': 'TOV',
    'player_points_rebounds_assists': 'PTS+REB+AST',
    'player_points_rebounds': 'PTS+REB',
    'player_points_assists': 'PTS+AST',
    'player_rebounds_assists': 'REB+AST',
    'player_blocks_steals': 'BLK+STL'
}

all_results = []

for category, stat in propDict.items():
    print(f"Processing {category}...")
    results = single_bet(data, all_bookmakers, category=category, stat_line=stat)
    all_results.append(results)

combined_results = pd.concat(all_results, ignore_index=True)

final_results = combined_results.sort_values(by='EV', ascending=False).reset_index(drop=True)

print("\nTop 15 highest EV bets across all prop types:")

# Save to CSV
final_results.to_csv(f'PROPS_EV/SingleBets({formatted_date}).csv')
final_results.head(15)

Processing player_points...
Processing player_rebounds...
Processing player_assists...
Processing player_threes...
Processing player_blocks...
Processing player_steals...
Processing player_field_goals...
Processing player_frees_made...
Processing player_frees_attempts...
Processing player_turnovers...
Processing player_points_rebounds_assists...
Processing player_points_rebounds...
Processing player_points_assists...
Processing player_rebounds_assists...
Processing player_blocks_steals...

Top 15 highest EV bets across all prop types:


,NAME,BOOKMAKER,CATEGORY,LINE,OVER/UNDER,ODDS,FAIR ODDS,SIM PROB,EV,KELLY CRITERION
0,Bennedict Mathurin,Bovada,player_points,9.5,Over,190,228,0.541,56.86,0.5385
1,Cason Wallace,Bovada,player_blocks,0.5,Over,235,277,0.464,55.49,0.4619
2,Cason Wallace,DraftKings,player_blocks,0.5,Over,230,277,0.462,52.33,0.4593
3,Cason Wallace,BetRivers,player_assists,1.5,Over,128,141,0.659,50.25,0.6563
4,Aaron Nesmith,DraftKings,player_blocks,0.5,Over,145,160,0.612,50.04,0.6097
5,Cason Wallace,DraftKings,player_blocks_steals,1.5,Over,150,174,0.599,49.70,0.5961
6,Aaron Wiggins,DraftKings,player_blocks,0.5,Over,370,441,0.318,49.69,0.3166
7,Tyrese Haliburton,Bovada,player_assists,7.5,Over,125,139,0.662,48.88,0.6590
8,Tyrese Haliburton,FanDuel,player_assists,7.5,Over,124,139,0.664,48.72,0.6612
9,Aaron Nesmith,Bovada,player_blocks,0.5,Over,145,160,0.603,47.84,0.6007


## PrizePicks & Underdog
## 2 leg EVs since we cant do single bets on those platforms

### Functions used to grab the ev and kelly

In [6]:
def prizePicksPairsEV(data, prizePicks, propDict, simulations=10000, stake=100, payout=300, weight=0.8):
    """Optimized version with pre-computed lookups and parallel processing"""
    
    # 1. Pre-compute player statistics for all stat lines
    print("Pre-computing player statistics...")
    player_stats_cache = {}
    
    for stat_line in set(propDict.values()):
        for player in data['PLAYER_NAME'].unique():
            stat_values = get_combined_stat_values(data, player, stat_line)
            if not stat_values.empty:
                mean, std = stat_values.mean(), stat_values.std()
                if std > 0:  # Only store valid stats
                    player_stats_cache[(player, stat_line)] = {
                        'mean': mean, 'std': std, 'values': stat_values
                    }
    
    # 2. Pre-compute prize picks lookups by category and player
    print("Pre-computing prize picks lookups...")
    prizepicks_lookup = {}
    for category in propDict.keys():
        category_data = prizePicks[prizePicks['CATEGORY'] == category]
        for _, row in category_data.iterrows():
            prizepicks_lookup[(row['NAME'], category)] = row['LINE']
    
    # 3. Pre-generate valid combinations (avoid duplicates upfront)
    print("Generating valid combinations...")
    valid_combinations = []
    
    available_players = set()
    for category in propDict.keys():
        category_players = prizePicks[prizePicks['CATEGORY'] == category]['NAME'].unique()
        for player in category_players:
            stat_line = propDict[category]
            if (player, stat_line) in player_stats_cache:
                available_players.add((player, category))
    
    available_players = list(available_players)
    
    # Generate all valid pairs
    for i in range(len(available_players)):
        for j in range(i + 1, len(available_players)):
            player1, cat1 = available_players[i]
            player2, cat2 = available_players[j]
            
            # Skip if same player
            if player1 == player2:
                continue
            
            # Check if all required data exists
            stat_line1, stat_line2 = propDict[cat1], propDict[cat2]
            
            if (player1, stat_line1) in player_stats_cache and \
               (player2, stat_line2) in player_stats_cache and \
               (player1, cat1) in prizepicks_lookup and \
               (player2, cat2) in prizepicks_lookup:
                
                valid_combinations.append({
                    'players': [player1, player2],
                    'categories': [cat1, cat2],
                    'stat_lines': [stat_line1, stat_line2],
                    'lines': [
                        prizepicks_lookup[(player1, cat1)],
                        prizepicks_lookup[(player2, cat2)]
                    ]
                })
    
    # 4. Process combinations with Monte Carlo simulations - THREADED VERSION
    print(f"Running Monte Carlo simulations for {len(valid_combinations)} combinations...")
    
    def process_combination(combo):
        """Process a single combination"""
        players, categories, stat_lines, lines = combo['players'], combo['categories'], combo['stat_lines'], combo['lines']
        
        try:
            # Get cached statistics
            stats = [player_stats_cache[(players[j], stat_lines[j])] for j in range(2)]
            means = [s['mean'] for s in stats]
            stds = [s['std'] for s in stats]
            
            # Generate simulations for each player individually
            sim1 = np.random.normal(means[0], stds[0], simulations)
            sim2 = np.random.normal(means[1], stds[1], simulations)
            
            # Calculate over/under results
            over1, under1 = sim1 > lines[0], sim1 <= lines[0]
            over2, under2 = sim2 > lines[1], sim2 <= lines[1]
            
            # Calculate Monte Carlo probabilities
            mc_probs = {
                'OVER/OVER': np.mean(over1 & over2),
                'UNDER/UNDER': np.mean(under1 & under2),
                'OVER/UNDER': np.mean(over1 & under2),
                'UNDER/OVER': np.mean(under1 & over2),
            }
            
            # Calculate z-score probabilities
            z_over1 = zscore_prob(means[0], stds[0], lines[0], side='over')
            z_under1 = 1 - z_over1
            z_over2 = zscore_prob(means[1], stds[1], lines[1], side='over')
            z_under2 = 1 - z_over2
            
            z_probs = {
                'OVER/OVER': z_over1 * z_over2,
                'UNDER/UNDER': z_under1 * z_under2,
                'OVER/UNDER': z_over1 * z_under2,
                'UNDER/OVER': z_under1 * z_over2,
            }
            
            # Combine probabilities
            final_probs = {
                k: weight * mc_probs[k] + (1 - weight) * z_probs[k]
                for k in mc_probs
            }
            
            # Calculate EVs
            evs = {k: round((final_probs[k] * payout) - stake, 2) for k in final_probs}
            
            # Find best combination
            best_type = max(evs, key=evs.get)
            best_ev = evs[best_type]
            best_prob = final_probs[best_type]
            
            return {
                'players': players,
                'categories': categories,
                'stat_lines': stat_lines,
                'lines': lines,
                'best_type': best_type,
                'best_ev': best_ev,
                'best_prob': best_prob
            }
            
        except Exception as e:
            print(f"Error processing combination {players}: {e}")
            return None
    
    # Process combinations in parallel using threads
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import multiprocessing as mp
    
    results = []
    max_workers = min(mp.cpu_count(), len(valid_combinations))
    
    print(f"Processing {len(valid_combinations)} combinations with {max_workers} threads...")
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all jobs
        future_to_combo = {
            executor.submit(process_combination, combo): i 
            for i, combo in enumerate(valid_combinations)
        }
        
        # Collect results as they complete
        completed = 0
        for future in as_completed(future_to_combo):
            try:
                result = future.result()
                if result is not None:
                    results.append(result)
                completed += 1
                
                # Show progress every 100 completions
                if completed % 100 == 0:
                    print(f"Completed {completed}/{len(valid_combinations)} combinations")
                    
            except Exception as e:
                print(f"Error in future: {e}")
    
    print(f"Successfully processed {len(results)} combinations")
    
    # 5. Build final results
    print("Building final results...")
    all_pairs = []
    
    for result in results:
        all_pairs.append({
            'PLAYER 1': result['players'][0],
            'CATEGORY 1': result['stat_lines'][0], 
            'PLAYER 1 LINE': result['lines'][0],
            'PLAYER 2': result['players'][1],
            'CATEGORY 2': result['stat_lines'][1],
            'PLAYER 2 LINE': result['lines'][1],
            'TYPE': result['best_type'],
            'EV': round(result['best_ev'], 2),
            'PROBABILITY': round(result['best_prob'], 4),
            'KELLY CRITERION': kelly_criterion(result['best_prob'], payout, stake)
        })
    
    return pd.DataFrame(all_pairs)

In [7]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
pp_data = pd.read_csv(f'PROPS_DATA/Playoffs_DFS({formatted_date}).csv')
prizePicks = pp_data[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[pp_data['BOOKMAKER'] == 'PrizePicks']
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_threes': 'FG3M',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_field_goals': 'FGM',
    'player_threes': 'FG3M',
    'player_frees_made': 'FTM',
    'player_frees_attempts': 'FTA',
    'player_points_rebounds_assists': 'PTS+REB+AST',
    'player_points_rebounds': 'PTS+REB',
    'player_points_assists': 'PTS+AST',
    'player_rebounds_assists': 'REB+AST',
    'player_turnovers': 'TOV',
}

# Run the analysis
results = prizePicksPairsEV(data, pp_data, propDict)
results.to_csv(f'PROPS_EV/PrizePicksPairs({formatted_date}).csv')

results.sort_values(by='KELLY CRITERION', ascending=False).head(15)


Pre-computing player statistics...
Pre-computing prize picks lookups...
Generating valid combinations...
Running Monte Carlo simulations for 9178 combinations...
Processing 9178 combinations with 8 threads...
Completed 100/9178 combinations
Completed 200/9178 combinations
Completed 300/9178 combinations
Completed 400/9178 combinations
Completed 500/9178 combinations
Completed 600/9178 combinations
Completed 700/9178 combinations
Completed 800/9178 combinations
Completed 900/9178 combinations
Completed 1000/9178 combinations
Completed 1100/9178 combinations
Completed 1200/9178 combinations
Completed 1300/9178 combinations
Completed 1400/9178 combinations
Completed 1500/9178 combinations
Completed 1600/9178 combinations
Completed 1700/9178 combinations
Completed 1800/9178 combinations
Completed 1900/9178 combinations
Completed 2000/9178 combinations
Completed 2100/9178 combinations
Completed 2200/9178 combinations
Completed 2300/9178 combinations
Completed 2400/9178 combinations
Complete

,PLAYER 1,CATEGORY 1,PLAYER 1 LINE,PLAYER 2,CATEGORY 2,PLAYER 2 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
7449,Cason Wallace,PTS+AST,5.5,Tyrese Haliburton,REB+AST,11.0,OVER/OVER,79.61,0.5987,0.5967
7543,Tyrese Haliburton,REB+AST,11.0,Isaiah Hartenstein,PTS+REB,12.0,OVER/OVER,74.62,0.5821,0.5800
329,Pascal Siakam,REB,7.5,Cason Wallace,PTS+AST,5.5,UNDER/OVER,74.61,0.5820,0.5799
7523,Tyrese Haliburton,REB+AST,11.0,Cason Wallace,REB+AST,3.5,OVER/OVER,73.78,0.5793,0.5772
7487,Cason Wallace,PTS+AST,5.5,Isaiah Hartenstein,PTS+REB,12.0,OVER/OVER,73.42,0.5781,0.5760
4360,Cason Wallace,PTS+REB+AST,8.0,Tyrese Haliburton,REB+AST,11.0,OVER/OVER,73.06,0.5769,0.5748
3776,Isaiah Hartenstein,PTS+REB+AST,14.0,Cason Wallace,PTS+AST,5.5,OVER/OVER,72.82,0.5761,0.5740
330,Pascal Siakam,REB,7.5,Tyrese Haliburton,REB+AST,11.0,UNDER/OVER,72.57,0.5752,0.5731
6573,Isaiah Hartenstein,PTS,5.5,Cason Wallace,PTS+AST,5.5,OVER/OVER,72.57,0.5752,0.5731
354,Pascal Siakam,REB,7.5,Cason Wallace,REB+AST,3.5,UNDER/OVER,71.28,0.5709,0.5688


## 3 leg EVs for prizepicks and underdogs

In [8]:

def prizePicksTriosEV(data, prizePicks, propDict, simulations=10000, stake=100, payout=600, weight=0.8):
    """Optimized version with pre-computed lookups and parallel processing"""
    
    # 1. Pre-compute player statistics for all stat lines
    print("Pre-computing player statistics...")
    player_stats_cache = {}
    
    for stat_line in set(propDict.values()):
        for player in data['PLAYER_NAME'].unique():
            stat_values = get_combined_stat_values(data, player, stat_line)
            if not stat_values.empty:
                mean, std = stat_values.mean(), stat_values.std()
                if std > 0:  # Only store valid stats
                    player_stats_cache[(player, stat_line)] = {
                        'mean': mean, 'std': std, 'values': stat_values
                    }
    
    # 2. Pre-compute prize picks lookups by category and player
    print("Pre-computing prize picks lookups...")
    prizepicks_lookup = {}
    for category in propDict.keys():
        category_data = prizePicks[prizePicks['CATEGORY'] == category]
        for _, row in category_data.iterrows():
            prizepicks_lookup[(row['NAME'], category)] = row['LINE']
    
    # 3. Pre-generate valid combinations (avoid duplicates upfront)
    print("Generating valid combinations...")
    valid_combinations = []
    
    available_players = set()
    for category in propDict.keys():
        category_players = prizePicks[prizePicks['CATEGORY'] == category]['NAME'].unique()
        for player in category_players:
            stat_line = propDict[category]
            if (player, stat_line) in player_stats_cache:
                available_players.add((player, category))
    
    available_players = list(available_players)
    
    # Generate all valid trios
    for i in range(len(available_players)):
        for j in range(i + 1, len(available_players)):
            for k in range(j + 1, len(available_players)):
                player1, cat1 = available_players[i]
                player2, cat2 = available_players[j]
                player3, cat3 = available_players[k]
                
                # Skip if any players are the same
                if player1 == player2 or player1 == player3 or player2 == player3:
                    continue
                
                # Check if all required data exists
                stat_line1, stat_line2, stat_line3 = propDict[cat1], propDict[cat2], propDict[cat3]
                
                if (player1, stat_line1) in player_stats_cache and \
                   (player2, stat_line2) in player_stats_cache and \
                   (player3, stat_line3) in player_stats_cache and \
                   (player1, cat1) in prizepicks_lookup and \
                   (player2, cat2) in prizepicks_lookup and \
                   (player3, cat3) in prizepicks_lookup:
                    
                    valid_combinations.append({
                        'players': [player1, player2, player3],
                        'categories': [cat1, cat2, cat3],
                        'stat_lines': [stat_line1, stat_line2, stat_line3],
                        'lines': [
                            prizepicks_lookup[(player1, cat1)],
                            prizepicks_lookup[(player2, cat2)],
                            prizepicks_lookup[(player3, cat3)]
                        ]
                    })
    
    # 4. Process combinations with Monte Carlo simulations - THREADED VERSION
    print(f"Running Monte Carlo simulations for {len(valid_combinations)} combinations...")
    
    def process_combination(combo):
        """Process a single combination"""
        players, categories, stat_lines, lines = combo['players'], combo['categories'], combo['stat_lines'], combo['lines']
        
        try:
            # Get cached statistics
            stats = [player_stats_cache[(players[j], stat_lines[j])] for j in range(3)]
            means = [s['mean'] for s in stats]
            stds = [s['std'] for s in stats]
            
            # Generate simulations for each player individually
            sim1 = np.random.normal(means[0], stds[0], simulations)
            sim2 = np.random.normal(means[1], stds[1], simulations)
            sim3 = np.random.normal(means[2], stds[2], simulations)
            
            # Calculate over/under results
            over1, under1 = sim1 > lines[0], sim1 <= lines[0]
            over2, under2 = sim2 > lines[1], sim2 <= lines[1]
            over3, under3 = sim3 > lines[2], sim3 <= lines[2]
            
            # Calculate Monte Carlo probabilities
            mc_probs = {
                'OVER/OVER/OVER': np.mean(over1 & over2 & over3),
                'OVER/OVER/UNDER': np.mean(over1 & over2 & under3),
                'OVER/UNDER/OVER': np.mean(over1 & under2 & over3),
                'OVER/UNDER/UNDER': np.mean(over1 & under2 & under3),
                'UNDER/OVER/OVER': np.mean(under1 & over2 & over3),
                'UNDER/OVER/UNDER': np.mean(under1 & over2 & under3),
                'UNDER/UNDER/OVER': np.mean(under1 & under2 & over3),
                'UNDER/UNDER/UNDER': np.mean(under1 & under2 & under3),
            }
            
            # Calculate z-score probabilities
            z_over1 = zscore_prob(means[0], stds[0], lines[0], side='over')
            z_under1 = 1 - z_over1
            z_over2 = zscore_prob(means[1], stds[1], lines[1], side='over')
            z_under2 = 1 - z_over2
            z_over3 = zscore_prob(means[2], stds[2], lines[2], side='over')
            z_under3 = 1 - z_over3
            
            z_probs = {
                'OVER/OVER/OVER': z_over1 * z_over2 * z_over3,
                'OVER/OVER/UNDER': z_over1 * z_over2 * z_under3,
                'OVER/UNDER/OVER': z_over1 * z_under2 * z_over3,
                'OVER/UNDER/UNDER': z_over1 * z_under2 * z_under3,
                'UNDER/OVER/OVER': z_under1 * z_over2 * z_over3,
                'UNDER/OVER/UNDER': z_under1 * z_over2 * z_under3,
                'UNDER/UNDER/OVER': z_under1 * z_under2 * z_over3,
                'UNDER/UNDER/UNDER': z_under1 * z_under2 * z_under3,
            }
            
            # Combine probabilities
            final_probs = {
                k: weight * mc_probs[k] + (1 - weight) * z_probs[k]
                for k in mc_probs
            }
            
            # Calculate EVs
            evs = {k: round((final_probs[k] * payout) - stake, 2) for k in final_probs}
            
            # Find best combination
            best_type = max(evs, key=evs.get)
            best_ev = evs[best_type]
            best_prob = final_probs[best_type]
            
            return {
                'players': players,
                'categories': categories,
                'stat_lines': stat_lines,
                'lines': lines,
                'best_type': best_type,
                'best_ev': best_ev,
                'best_prob': best_prob
            }
            
        except Exception as e:
            print(f"Error processing combination {players}: {e}")
            return None
    
    # Process combinations in parallel using threads
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import multiprocessing as mp
    
    results = []
    max_workers = min(mp.cpu_count(), len(valid_combinations))
    
    print(f"Processing {len(valid_combinations)} combinations with {max_workers} threads...")
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all jobs
        future_to_combo = {
            executor.submit(process_combination, combo): i 
            for i, combo in enumerate(valid_combinations)
        }
        
        # Collect results as they complete
        completed = 0
        for future in as_completed(future_to_combo):
            try:
                result = future.result()
                if result is not None:
                    results.append(result)
                completed += 1
                
                # Show progress every 100 completions
                if completed % 100 == 0:
                    print(f"Completed {completed}/{len(valid_combinations)} combinations")
                    
            except Exception as e:
                print(f"Error in future: {e}")
    
    print(f"Successfully processed {len(results)} combinations")
    
    # 5. Build final results
    print("Building final results...")
    all_trios = []
    
    for result in results:
        all_trios.append({
            'PLAYER 1': result['players'][0],
            'CATEGORY 1': result['stat_lines'][0], 
            'PLAYER 1 LINE': result['lines'][0],
            'PLAYER 2': result['players'][1],
            'CATEGORY 2': result['stat_lines'][1],
            'PLAYER 2 LINE': result['lines'][1],
            'PLAYER 3': result['players'][2],
            'CATEGORY 3': result['stat_lines'][2],
            'PLAYER 3 LINE': result['lines'][2],
            'TYPE': result['best_type'],
            'EV': round(result['best_ev'], 2),
            'PROBABILITY': round(result['best_prob'], 4),
            'KELLY CRITERION': kelly_criterion(result['best_prob'], payout, stake)
        })
    
    return pd.DataFrame(all_trios)

In [9]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
prizePicks = pd.read_csv(f'PROPS_DATA/Playoffs_DFS({formatted_date}).csv')
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_threes': 'FG3M',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_field_goals': 'FGM',
    'player_threes': 'FG3M',
    'player_frees_made': 'FTM',
    'player_frees_attempts': 'FTA',
    'player_points_rebounds_assists': 'PTS+REB+AST',
    'player_points_rebounds': 'PTS+REB',
    'player_points_assists': 'PTS+AST',
    'player_rebounds_assists': 'REB+AST',
    'player_turnovers': 'TOV',
}

trio_results = prizePicksTriosEV(data, prizePicks, propDict, payout=600).sort_values('KELLY CRITERION', ascending=False).reset_index(drop=True)
trio_results.to_csv(f'PROPS_EV/PrizePicksTrios({formatted_date}).csv')
trio_results

Pre-computing player statistics...
Pre-computing prize picks lookups...
Generating valid combinations...
Running Monte Carlo simulations for 374162 combinations...
Processing 374162 combinations with 8 threads...
Completed 100/374162 combinations
Completed 200/374162 combinations
Completed 300/374162 combinations
Completed 400/374162 combinations
Completed 500/374162 combinations
Completed 600/374162 combinations
Completed 700/374162 combinations
Completed 800/374162 combinations
Completed 900/374162 combinations
Completed 1000/374162 combinations
Completed 1100/374162 combinations
Completed 1200/374162 combinations
Completed 1300/374162 combinations
Completed 1400/374162 combinations
Completed 1500/374162 combinations
Completed 1600/374162 combinations
Completed 1700/374162 combinations
Completed 1800/374162 combinations
Completed 1900/374162 combinations
Completed 2000/374162 combinations
Completed 2100/374162 combinations
Completed 2200/374162 combinations
Completed 2300/374162 comb

,PLAYER 1,CATEGORY 1,PLAYER 1 LINE,PLAYER 2,CATEGORY 2,PLAYER 2 LINE,PLAYER 3,CATEGORY 3,PLAYER 3 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
0,Cason Wallace,PTS+AST,5.5,Tyrese Haliburton,REB+AST,11.0,Isaiah Hartenstein,PTS+REB,12.0,OVER/OVER/OVER,175.98,0.4600,0.4589
1,Pascal Siakam,REB,7.5,Cason Wallace,PTS+AST,5.5,Tyrese Haliburton,REB+AST,11.0,UNDER/OVER/OVER,169.22,0.4487,0.4476
2,Isaiah Hartenstein,REB,6.0,Cason Wallace,PTS+AST,5.5,Tyrese Haliburton,REB+AST,11.0,OVER/OVER/OVER,166.50,0.4442,0.4431
3,Tyrese Haliburton,REB+AST,11.0,Cason Wallace,REB+AST,3.5,Isaiah Hartenstein,PTS+REB,12.0,OVER/OVER/OVER,166.27,0.4438,0.4427
4,Isaiah Hartenstein,PTS,5.5,Cason Wallace,PTS+AST,5.5,Tyrese Haliburton,REB+AST,11.0,OVER/OVER/OVER,166.03,0.4434,0.4423
...,...,...,...,...,...,...,...,...,...,...,...,...,...
374157,Jalen Williams,PTS+REB+AST,32.5,Kenrich Williams,AST,0.5,Chet Holmgren,FGM,5.5,OVER/OVER/OVER,-23.21,0.1280,0.1262
374158,Kenrich Williams,AST,0.5,Alex Caruso,PTS+REB,12.5,Chet Holmgren,FGM,5.5,OVER/UNDER/OVER,-23.34,0.1278,0.1260
374159,Shai Gilgeous-Alexander,FTA,9.5,Chet Holmgren,PTS+AST,16.5,Jalen Williams,PTS+AST,27.0,UNDER/UNDER/UNDER,-23.50,0.1275,0.1258
374160,Pascal Siakam,PTS+AST,24.5,Kenrich Williams,AST,0.5,Chet Holmgren,FGM,5.5,OVER/OVER/OVER,-23.63,0.1273,0.1255


⚙️ Hybrid Approach Recommended
For NBA prop betting, a hybrid workflow often works best:
- Train an ML model to predict the expected prop value using features like usage rate, matchup context, rest, etc.
- Simulate multiple game scenarios via Monte Carlo, using your ML model’s predictive distribution (e.g., add residual noise or use a Bayesian framework).
- Bootstrap on your historical residuals to calibrate confidence intervals around those predictions.

This combines:
ML’s ability to capture complex relationships,
Monte Carlo’s capacity for modeling variability and outputting a full prob. distribution,
Bootstrap’s non-parametric estimate of uncertainty—all necessary to assess value and risk correctly 

🗓️ Practical Takeaway
Need a full probability distribution for value + risk? MC sim driven by your ML model is essential.


📌 Final Recommendation
Use Machine Learning to predict the mean prop, then Monte Carlo to simulate possible outcomes (using those ML predictions with added variance), and Bootstrap to quantify your confidence in those outcomes. This hybrid setup offers the most accurate, actionable probabilities for NBA prop betting.
